In [31]:
import os
import math
import numpy as np
import pandas as pd
import re

In [70]:
dataset = 'iconqa' # 'scienceqa' # 

In [71]:
paths = os.walk(f'../input/results/{dataset}')
paths

for path in paths:
    print(path)

('../input/results/iconqa', ['iconqa_1', 'iconqa_17', 'iconqa_2', 'iconqa_18', 'iconqa_6', 'iconqa_5', 'iconqa_21', 'iconqa_22', 'iconqa_10', 'iconqa_9', 'iconqa_25', 'iconqa_26', 'iconqa_30', 'iconqa_4', 'iconqa_8', 'iconqa_12', 'iconqa_14', 'iconqa_13', 'iconqa_29', 'iconqa_3', 'iconqa_7', 'iconqa_11', 'iconqa_15', 'iconqa_19', 'iconqa_16', 'iconqa_20', 'iconqa_27', 'iconqa_31', 'iconqa_32', 'iconqa_23', 'iconqa_24', 'iconqa_28'], [])
('../input/results/iconqa/iconqa_1', ['20240328163', '20240328165', '20240403171', '20240403173_good'], [])
('../input/results/iconqa/iconqa_1/20240328163', ['result'], ['log.txt'])
('../input/results/iconqa/iconqa_1/20240328163/result', [], [])
('../input/results/iconqa/iconqa_1/20240328165', ['result'], ['log.txt'])
('../input/results/iconqa/iconqa_1/20240328165/result', [], [])
('../input/results/iconqa/iconqa_1/20240403171', ['result'], ['log.txt'])
('../input/results/iconqa/iconqa_1/20240403171/result', [], [])
('../input/results/iconqa/iconqa_1/20

In [72]:
def get_total_training_time(lines):
    total_sec = 0
    pattern = re.compile(r'Train: data epoch: \[\d+\] Total time:')
    sub_lines = [pattern.sub('', x) for x in lines if pattern.match(x)]
    sub_lines = [x.strip().split(' ')[0] for x in sub_lines]
    for time_str in sub_lines:
        h ,m, s = [int(x) for x in time_str.split(':')]
        total_sec = total_sec + s + m*60 + h*3600
    return total_sec/3600

def get_training_time_per_it(lines):
    total_sec = []
    pattern = re.compile(r'Train: data epoch: \[\d+\] Total time:')
    sub_lines = [pattern.sub('', x) for x in lines if pattern.match(x)]
    sub_lines = [x.strip().split(' ')[1] for x in sub_lines]
    for time_str in sub_lines:
        sec = float(time_str.replace('(', ''))
        total_sec.append(sec)
    return sum(total_sec) / len(total_sec) 

def get_total_eval_time(lines):
    total_sec = 0
    pattern = re.compile(r'Evaluation Total time:')
    sub_lines = [pattern.sub('', x) for x in lines if pattern.match(x)]
    sub_lines = [x.strip().split(' ')[0] for x in sub_lines]
    for time_str in sub_lines:
        h ,m, s = [int(x) for x in time_str.split(':')]
        total_sec = total_sec + s + m*60 + h*3600
    return total_sec/3600

def get_llm_trainable_params(lines):
    # line example: trainable params: 1,032,192 || all params: 2,850,789,376 || trainable%: 0.03620723469400217
    sub_lines = [x.lower() for x in lines if x.lower().startswith('trainable params: ')] 
    sub_lines = [x.split(' ')[2] for x in sub_lines]
    if len(sub_lines) > 0:
        return int(sub_lines[0].replace(',', ''))
    else:
        return 0

def get_qformer_trainable_params(lines):
    sub_lines = [x.lower() for x in lines if x.lower().startswith('number of trainable parameters in qformer:')] 
    sub_lines = [x.split(' ')[-1] for x in sub_lines]
    if len(sub_lines) > 0:
        return int(sub_lines[0].replace(',', ''))
    else:
        return 0

def get_loss_hist(lines):
    sub_lines = [x.lower() for x in lines if 'loss: ' in x.lower()] 
    sub_lines = [(x.split('loss: ')[1]).split(' ')[0] for x in sub_lines]
    sub_lines = [float(x) for x in sub_lines]
    return sub_lines


def get_loss_per_epoch(lines):
    # example: {"train_lr": "0.000", "train_loss": "0.102"}
    sub_lines = [x.lower() for x in lines if '"train_loss":' in x.lower()]
    sub_lines = [(x.replace('"', '').replace('}', '').split(' ')[3]) for x in sub_lines]
    sub_lines = [float(x) for x in sub_lines]
    return sub_lines

def get_val_acc_per_epoch(lines):
    # example: {"val_agg_metrics": 77.30090605627086, "val_acc": 77.30090605627086, "val_best_epoch": 0}
    sub_lines = [x.lower() for x in lines if '"val_acc":' in x.lower()]
    sub_lines = [(x.replace('"', '').replace(',', '').split(' ')[3]) for x in sub_lines]
    sub_lines = [float(x) for x in sub_lines]
    return sub_lines

In [73]:
paths = os.walk(f'../input/results/{dataset}')

result_list = []
for path in paths:
    sub_dir = path[0]
    files = path[2]
    if sub_dir.endswith('_good'):
        test_result_file = f'test_{dataset}_result.json'
        val_result_file = f'val_{dataset}_result.json'
        experiment_id = (sub_dir.split('/')[4]).split('_')[1]
        
        test_df = pd.read_json(f'{sub_dir}/result/{test_result_file}')
        test_df['correct'] = test_df['pred_ans'] == test_df['gt_ans']
        test_accuracy = test_df['correct'].sum()/test_df.shape[0]
        # save the wrong questions
        test_df[test_df['correct'] == False].to_csv(f'{sub_dir}/result/wrong_questions.csv', index=False)

        val_df = pd.read_json(f'{sub_dir}/result/{val_result_file}')
        val_df['correct'] = val_df['pred_ans'] == val_df['gt_ans']
        val_accuracy = val_df['correct'].sum()/val_df.shape[0]
    
        log_file = [x for x in files if x.endswith('.log')][0]
        with open(f'{sub_dir}/{log_file}') as f:
            lines = f.readlines()
            training_time = get_total_training_time(lines)
            time_per_it = get_training_time_per_it(lines)
            eval_time = get_total_eval_time(lines)
            llm_trainable_params = get_llm_trainable_params(lines)
            qformer_trainable_params = get_qformer_trainable_params(lines)
            loss_hist = get_loss_hist(lines)
        with open(f'{sub_dir}/log.txt') as f:
            lines = f.readlines()
            loss_per_epoch = get_loss_per_epoch(lines)
            val_acc_per_epoch = get_val_acc_per_epoch(lines)

        
        result_list.append({
            'experiment_id': int(experiment_id),
            'test_accuracy': test_accuracy,
            'val_accuracy': val_accuracy,
            'training_time': training_time,
            'time_per_it': time_per_it,
            'eval_time': eval_time,
            'llm_trainable_params': llm_trainable_params,
            'qformer_trainable_params': qformer_trainable_params,
            'loss_hist': loss_hist,
            'loss_per_epoch': loss_per_epoch,
            'val_acc_per_epoch': val_acc_per_epoch
        })

In [74]:
result_df = pd.DataFrame(result_list)
result_df.sort_values(by=['experiment_id'])
# result_df.sort_values(by=['training_time'])

,experiment_id,test_accuracy,val_accuracy,training_time,time_per_it,eval_time,llm_trainable_params,qformer_trainable_params,loss_hist,loss_per_epoch,val_acc_per_epoch
0,1,0.742717,0.723243,5.903056,0.408009,2.597778,1032192,0,"[0.6991, 0.3073, 0.1862, 0.107, 0.3492, 0.2191...","[0.154, 0.116, 0.098, 0.089, 0.082, 0.079, 0.0...","[66.03863204559848, 68.84103863204561, 68.5877..."
2,2,0.739234,0.722926,5.466667,0.415620,2.360833,2064384,0,"[0.7105, 0.3305, 0.236, 0.1027, 0.391, 0.2135,...","[0.156, 0.113, 0.096, 0.087, 0.081, 0.078, 0.0...","[67.19442685243826, 68.1760607979734, 69.49018..."
19,3,0.753167,0.746992,8.102500,0.410707,3.463056,4128768,0,"[0.7188, 0.2298, 0.1928, 0.1648, 0.3075, 0.207...","[0.153, 0.115, 0.1, 0.09, 0.082, 0.079, 0.075,...","[66.07029765674477, 66.2444585180494, 67.47941..."
13,4,0.729417,0.733851,4.274722,0.406250,1.930278,8257536,0,"[0.7184, 0.2349, 0.2304, 0.1273, 0.3047, 0.242...","[0.156, 0.117, 0.099, 0.088, 0.082, 0.077, 0.0...","[66.2444585180494, 67.30525649145028, 70.55098..."
5,5,0.747625,0.725617,8.079167,0.409500,3.406667,589824,0,"[0.7223, 0.2497, 0.2203, 0.1119, 0.2821, 0.225...","[0.159, 0.117, 0.1, 0.09, 0.084, 0.079, 0.076,...","[63.01456618112729, 67.32108929702343, 68.0018..."
4,6,0.729101,0.718493,5.465000,0.415510,2.347222,1179648,0,"[0.704, 0.2716, 0.1871, 0.1251, 0.3121, 0.2337...","[0.155, 0.114, 0.097, 0.088, 0.081, 0.076, 0.0...","[65.83280557314755, 67.59024699176695, 69.5851..."
20,7,0.743509,0.735592,6.895278,0.403292,2.928056,2359296,0,"[0.7194, 0.2532, 0.2272, 0.1145, 0.3076, 0.241...","[0.154, 0.113, 0.094, 0.087, 0.082, 0.077, 0.0...","[64.70867637745408, 67.25775807473084, 68.2552..."
14,8,0.727834,0.714218,5.869722,0.405700,2.586389,4718592,0,"[0.7105, 0.2472, 0.2146, 0.1434, 0.282, 0.2245...","[0.159, 0.117, 0.1, 0.091, 0.084, 0.079, 0.076...","[65.84863837872071, 67.2102596580114, 68.90436..."
9,9,0.745408,0.737017,7.210000,0.456825,3.149722,1622016,0,"[0.7395, 0.2336, 0.2247, 0.1168, 0.3086, 0.212...","[0.158, 0.115, 0.099, 0.088, 0.082, 0.079, 0.0...","[66.95693476884104, 66.35528815706144, 70.4243..."
8,10,0.757283,0.752850,9.097778,0.461147,3.838333,3244032,0,"[0.7194, 0.264, 0.1849, 0.1432, 0.3459, 0.2349...","[0.151, 0.112, 0.095, 0.087, 0.08, 0.077, 0.07...","[67.79607346421786, 68.99936668777707, 68.8568..."


In [75]:
method_option_list = [
    "LLM LoRA (ffn, FlanT5-XL)             ",
    "LLM LoRA (attn, FlanT5-XL)            ",
    "LLM LoRA (all, FlanT5-XL)             ",
    "Q-Former LoRA (ffn, FlanT5-XL)        ",
    "Q-Former LoRA (self-attn, FlanT5-XL)  ",
    "Q-Former LoRA (cross-attn, FlanT5-XL) ",
    "Q-Former LoRA (all, FlanT5-XL)        ",
    "Q-Former and LLM LoRA (all, FlanT5-XL)",
    "LLM LoRA (ffn, Vicuna-7B)             ",
    "LLM LoRA (attn, Vicuna-7B)            ",
    "LLM LoRA (all, Vicuna-7B)             ",
    "Q-Former LoRA (ffn, Vicuna-7B)        ",
    "Q-Former LoRA (self-attn, Vicuna-7B)  ",
    "Q-Former LoRA (cross-attn, Vicuna-7B) ",
    "Q-Former LoRA (all, Vicuna-7B)        ",
    "Q-Former and LLM LoRA (all, Vicuna-7B)",
    "Q-Former (self-attn QVO, FlanT5-XL)   ",
    "Q-Former (cross-attn QVO, FlanT5-XL)  ",
    "Q-Former (cross-attn QV, FlanT5-XL)   ",
]
method_option_list = [x.strip() for x in method_option_list]

r_option_list = [
    "r = 1",
    "r = 2",
    "r = 4",
    "r = 8",
]

def experiment_id_to_info(id):
    row = math.floor((id-1)/4)
    col = (id-1)%4
    method = ""
    r = 0

    method_option = method_option_list[row]

    r_option = r_option_list[col]

    return (method_option, r_option)

experiment_id_to_info(7)

('LLM LoRA (attn, FlanT5-XL)', 'r = 4')

In [76]:
test_accuracy_df = pd.DataFrame(np.zeros((19,4)), columns=r_option_list, index=method_option_list)
val_accuracy_df = test_accuracy_df.copy()
training_time_df = test_accuracy_df.copy()
time_per_it_df = test_accuracy_df.copy()
trainable_params_df = test_accuracy_df.copy()
loss_hist_df = test_accuracy_df.copy()
loss_per_epoch_df = test_accuracy_df.copy()
val_acc_per_epoch_df = test_accuracy_df.copy()

for result in result_list:
    id = result['experiment_id']
    
    method_option, r_option = experiment_id_to_info(id)
    test_accuracy_df.at[method_option, r_option] = result['test_accuracy']
    val_accuracy_df.at[method_option, r_option] = result['val_accuracy']
    training_time_df.at[method_option, r_option] = result['training_time']
    time_per_it_df.at[method_option, r_option] = result['time_per_it']
    trainable_params_df.at[method_option, r_option] = result['llm_trainable_params'] + result['qformer_trainable_params']
    loss_hist_df.at[method_option, r_option] = str(result['loss_hist'])
    loss_per_epoch_df.at[method_option, r_option] = str(result['loss_per_epoch'])
    val_acc_per_epoch_df.at[method_option, r_option] = str(result['val_acc_per_epoch'])


/tmp/ipykernel_93042/2001529989.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.6991, 0.3073, 0.1862, 0.107, 0.3492, 0.2191, 0.1775, 0.1711, 0.1743, 0.2838, 0.3249, 0.1213, 0.1005, 0.1618, 0.1851, 0.1774, 0.177, 0.2357, 0.1534, 0.138, 0.1422, 0.1951, 0.1165, 0.1824, 0.1343, 0.0693, 0.2148, 0.1615, 0.0941, 0.1355, 0.0926, 0.1492, 0.141, 0.508, 0.1396, 0.1068, 0.0748, 0.14, 0.118, 0.3363, 0.118, 0.2328, 0.1089, 0.142, 0.1336, 0.231, 0.0578, 0.2556, 0.2198, 0.1064, 0.1288, 0.1763, 0.0861, 0.1735, 0.0808, 0.3305, 0.1247, 0.2025, 0.1497, 0.1202, 0.1427, 0.1574, 0.0197, 0.0562, 0.1179, 0.0997, 0.1761, 0.1483, 0.0872, 0.0686, 0.1491, 0.0367, 0.2538, 0.0921, 0.1241, 0.1312, 0.0717, 0.1513, 0.0376, 0.1207, 0.1346, 0.089, 0.1905, 0.1751, 0.148, 0.0434, 0.1328, 0.2504, 0.0555, 0.0975, 0.0911, 0.2544, 0.02, 0.1466, 0.1598, 0.0306, 0.1544, 0.0545, 0.0939, 0.0633, 0.0755, 0.1267, 0.1378, 0.1686, 0.1139, 0.1741

In [43]:
test_accuracy_df.to_csv(f'{dataset}/test_accuracy.csv', index=True)
test_accuracy_df

,r = 1,r = 2,r = 4,r = 8
"LLM LoRA (ffn, FlanT5-XL)",0.872583,0.852256,0.863659,0.865146
"LLM LoRA (attn, FlanT5-XL)",0.862172,0.848785,0.849281,0.856718
"LLM LoRA (all, FlanT5-XL)",0.854239,0.864155,0.856222,0.856718
"Q-Former LoRA (ffn, FlanT5-XL)",0.844819,0.844819,0.844819,0.844819
"Q-Former LoRA (self-attn, FlanT5-XL)",0.843332,0.843332,0.801686,0.843332
"Q-Former LoRA (cross-attn, FlanT5-XL)",0.847298,0.809122,0.847298,0.847298
"Q-Former LoRA (all, FlanT5-XL)",0.815568,0.815568,0.815568,0.815568
"Q-Former and LLM LoRA (all, FlanT5-XL)",0.872583,0.861180,0.870104,0.862667
"LLM LoRA (ffn, Vicuna-7B)",0.355478,0.355478,0.355478,0.355478
"LLM LoRA (attn, Vicuna-7B)",0.355478,0.355478,0.355478,0.355478


In [19]:
val_accuracy_df.to_csv(f'{dataset}/val_accuracy.csv', index=True)
val_accuracy_df

,r = 1,r = 2,r = 4,r = 8
"LLM LoRA (ffn, FlanT5-XL)",0.853600,0.826419,0.834526,0.834526
"LLM LoRA (attn, FlanT5-XL)",0.829757,0.834049,0.833095,0.832141
"LLM LoRA (all, FlanT5-XL)",0.828326,0.847401,0.839771,0.829757
"Q-Former LoRA (ffn, FlanT5-XL)",0.824988,0.824988,0.824988,0.824988
"Q-Former LoRA (self-attn, FlanT5-XL)",0.827849,0.827849,0.788269,0.827849
"Q-Former LoRA (cross-attn, FlanT5-XL)",0.824988,0.802575,0.824988,0.824988
"Q-Former LoRA (all, FlanT5-XL)",0.809251,0.809251,0.809251,0.809251
"Q-Former and LLM LoRA (all, FlanT5-XL)",0.850739,0.851216,0.856462,0.852170
"LLM LoRA (ffn, Vicuna-7B)",0.371006,0.371006,0.371006,0.371006
"LLM LoRA (attn, Vicuna-7B)",0.371006,0.371006,0.371006,0.371006


In [44]:
training_time_df.to_csv(f'{dataset}/training_time.csv', index=True)
training_time_df

,r = 1,r = 2,r = 4,r = 8
"LLM LoRA (ffn, FlanT5-XL)",3.658333,2.751944,3.770833,3.788611
"LLM LoRA (attn, FlanT5-XL)",2.425278,2.679722,2.674722,2.923889
"LLM LoRA (all, FlanT5-XL)",3.126389,3.912778,2.881111,3.175556
"Q-Former LoRA (ffn, FlanT5-XL)",3.606667,3.638889,3.643056,3.634167
"Q-Former LoRA (self-attn, FlanT5-XL)",2.804167,2.840833,2.611944,2.830278
"Q-Former LoRA (cross-attn, FlanT5-XL)",3.443611,3.438889,3.451389,3.432500
"Q-Former LoRA (all, FlanT5-XL)",1.813889,1.849444,1.848889,1.846667
"Q-Former and LLM LoRA (all, FlanT5-XL)",3.884722,3.280278,4.500278,3.882778
"LLM LoRA (ffn, Vicuna-7B)",0.968333,0.981667,0.979444,0.980278
"LLM LoRA (attn, Vicuna-7B)",0.939167,0.940556,0.942222,0.946111


In [45]:
time_per_it_df.to_csv(f'{dataset}/time_per_it.csv', index=True)
time_per_it_df

,r = 1,r = 2,r = 4,r = 8
"LLM LoRA (ffn, FlanT5-XL)",0.565313,0.579909,0.582727,0.585480
"LLM LoRA (attn, FlanT5-XL)",0.562190,0.564655,0.563664,0.564733
"LLM LoRA (all, FlanT5-XL)",0.603858,0.604720,0.607045,0.613300
"Q-Former LoRA (ffn, FlanT5-XL)",0.557320,0.562367,0.562940,0.561567
"Q-Former LoRA (self-attn, FlanT5-XL)",0.541683,0.548708,0.550345,0.546692
"Q-Former LoRA (cross-attn, FlanT5-XL)",0.532133,0.531407,0.533367,0.530473
"Q-Former LoRA (all, FlanT5-XL)",0.600543,0.612486,0.612314,0.611457
"Q-Former and LLM LoRA (all, FlanT5-XL)",0.692585,0.691209,0.695280,0.692246
"LLM LoRA (ffn, Vicuna-7B)",0.561125,0.568975,0.567650,0.568075
"LLM LoRA (attn, Vicuna-7B)",0.544150,0.544925,0.545925,0.548175


In [46]:
trainable_params_df.astype(int).to_csv(f'{dataset}/trainable_params.csv', index=True)
trainable_params_df.astype(int)

,r = 1,r = 2,r = 4,r = 8
"LLM LoRA (ffn, FlanT5-XL)",1032192,2064384,4128768,8257536
"LLM LoRA (attn, FlanT5-XL)",589824,1179648,2359296,4718592
"LLM LoRA (all, FlanT5-XL)",1622016,3244032,6488064,12976128
"Q-Former LoRA (ffn, FlanT5-XL)",184320,368640,737280,1474560
"Q-Former LoRA (self-attn, FlanT5-XL)",36864,73728,147456,294912
"Q-Former LoRA (cross-attn, FlanT5-XL)",44544,89088,178176,356352
"Q-Former LoRA (all, FlanT5-XL)",265728,531456,1062912,2125824
"Q-Former and LLM LoRA (all, FlanT5-XL)",1887744,3775488,7550976,15101952
"LLM LoRA (ffn, Vicuna-7B)",1449984,2899968,5799936,11599872
"LLM LoRA (attn, Vicuna-7B)",524288,1048576,2097152,4194304


In [47]:
loss_hist_df.to_csv(f'{dataset}/loss_hist.csv', index=True)
loss_hist_df

,r = 1,r = 2,r = 4,r = 8
"LLM LoRA (ffn, FlanT5-XL)","[0.2437, 0.1342, 0.1701, 0.1755, 0.1314, 0.052...","[0.2472, 0.1628, 0.1447, 0.1174, 0.1256, 0.041...","[0.2428, 0.1389, 0.1275, 0.1507, 0.1255, 0.047...","[0.2165, 0.1376, 0.1535, 0.0659, 0.1308, 0.046..."
"LLM LoRA (attn, FlanT5-XL)","[0.2394, 0.1274, 0.1313, 0.1086, 0.1423, 0.057...","[0.221, 0.149, 0.1634, 0.0836, 0.1367, 0.051, ...","[0.2467, 0.15, 0.1737, 0.078, 0.1414, 0.0486, ...","[0.2451, 0.1482, 0.1613, 0.136, 0.1171, 0.0473..."
"LLM LoRA (all, FlanT5-XL)","[0.2185, 0.1598, 0.1388, 0.069, 0.1234, 0.0407...","[0.2421, 0.1484, 0.1616, 0.0975, 0.1291, 0.049...","[0.2334, 0.1187, 0.1774, 0.1301, 0.1221, 0.042...","[0.2047, 0.1421, 0.1705, 0.0805, 0.1267, 0.039..."
"Q-Former LoRA (ffn, FlanT5-XL)","[0.2295, 0.1339, 0.1704, 0.0791, 0.1271, 0.043...","[0.2295, 0.1339, 0.1704, 0.0791, 0.1271, 0.043...","[0.2295, 0.1339, 0.1704, 0.0791, 0.1271, 0.043...","[0.2295, 0.1339, 0.1704, 0.0791, 0.1271, 0.043..."
"Q-Former LoRA (self-attn, FlanT5-XL)","[0.2207, 0.1499, 0.1777, 0.1229, 0.12, 0.0452,...","[0.2207, 0.1499, 0.1777, 0.1229, 0.12, 0.0452,...","[0.2207, 0.1855, 0.1669, 0.0937, 0.1488, 0.049...","[0.2207, 0.1499, 0.1777, 0.1229, 0.12, 0.0452,..."
"Q-Former LoRA (cross-attn, FlanT5-XL)","[0.2083, 0.1379, 0.1765, 0.0972, 0.1229, 0.046...","[0.2083, 0.1616, 0.2149, 0.0923, 0.1291, 0.049...","[0.2083, 0.1379, 0.1765, 0.0972, 0.1229, 0.046...","[0.2083, 0.1379, 0.1765, 0.0972, 0.1229, 0.046..."
"Q-Former LoRA (all, FlanT5-XL)","[0.2433, 0.1378, 0.1676, 0.125, 0.1292, 0.0475...","[0.2433, 0.1378, 0.1676, 0.125, 0.1292, 0.0475...","[0.2433, 0.1378, 0.1676, 0.125, 0.1292, 0.0475...","[0.2433, 0.1378, 0.1676, 0.125, 0.1292, 0.0475..."
"Q-Former and LLM LoRA (all, FlanT5-XL)","[0.2424, 0.1573, 0.1535, 0.1062, 0.1225, 0.043...","[0.2302, 0.1708, 0.2173, 0.0773, 0.1288, 0.043...","[0.242, 0.1736, 0.1802, 0.1149, 0.1158, 0.0471...","[0.2171, 0.1627, 0.1578, 0.1091, 0.1205, 0.044..."
"LLM LoRA (ffn, Vicuna-7B)","[0.1046, nan, nan, nan, nan, nan, nan, nan, na...","[0.0393, nan, nan, nan, nan, nan, nan, nan, na...","[0.0411, nan, nan, nan, nan, nan, nan, nan, na...","[0.0435, nan, nan, nan, nan, nan, nan, nan, na..."
"LLM LoRA (attn, Vicuna-7B)","[0.0822, nan, nan, nan, nan, nan, nan, nan, na...","[0.0926, nan, nan, nan, nan, nan, nan, nan, na...","[0.0406, nan, nan, nan, nan, nan, nan, nan, na...","[0.0945, nan, nan, nan, nan, nan, nan, nan, na..."


In [77]:
loss_per_epoch_df.to_csv(f'{dataset}/loss_per_epoch.csv', index=True)
loss_per_epoch_df

,r = 1,r = 2,r = 4,r = 8
"LLM LoRA (ffn, FlanT5-XL)","[0.154, 0.116, 0.098, 0.089, 0.082, 0.079, 0.0...","[0.156, 0.113, 0.096, 0.087, 0.081, 0.078, 0.0...","[0.153, 0.115, 0.1, 0.09, 0.082, 0.079, 0.075,...","[0.156, 0.117, 0.099, 0.088, 0.082, 0.077, 0.0..."
"LLM LoRA (attn, FlanT5-XL)","[0.159, 0.117, 0.1, 0.09, 0.084, 0.079, 0.076,...","[0.155, 0.114, 0.097, 0.088, 0.081, 0.076, 0.0...","[0.154, 0.113, 0.094, 0.087, 0.082, 0.077, 0.0...","[0.159, 0.117, 0.1, 0.091, 0.084, 0.079, 0.076..."
"LLM LoRA (all, FlanT5-XL)","[0.158, 0.115, 0.099, 0.088, 0.082, 0.079, 0.0...","[0.151, 0.112, 0.095, 0.087, 0.08, 0.077, 0.07...","[0.151, 0.114, 0.097, 0.089, 0.082, 0.077, 0.0...","[0.158, 0.115, 0.099, 0.089, 0.083, 0.077, 0.0..."
"Q-Former LoRA (ffn, FlanT5-XL)","[0.163, 0.133, 0.12, 0.112, 0.105, 0.101, 0.09...","[0.163, 0.133, 0.12, 0.112, 0.105, 0.101, 0.09...","[0.187, 0.154, 0.141, 0.136, 0.131, 0.125, 0.1...","[0.163, 0.133, 0.12, 0.112, 0.105, 0.101, 0.09..."
"Q-Former LoRA (self-attn, FlanT5-XL)","[0.164, 0.131, 0.12, 0.112, 0.105, 0.101, 0.09...","[0.164, 0.131, 0.12, 0.112, 0.105, 0.101, 0.09...","[0.164, 0.131, 0.12, 0.112, 0.105, 0.101, 0.09...","[0.164, 0.131, 0.12, 0.112, 0.105, 0.101, 0.09..."
"Q-Former LoRA (cross-attn, FlanT5-XL)","[0.168, 0.156, 0.14, 0.132, 0.139, 0.123, 0.11...","[0.168, 0.156, 0.14, 0.132, 0.139, 0.123, 0.11...","[0.168, 0.156, 0.14, 0.132, 0.139, 0.123, 0.11...","[0.168, 0.156, 0.14, 0.132, 0.139, 0.123, 0.11..."
"Q-Former LoRA (all, FlanT5-XL)","[0.166, 0.133, 0.121, 0.112, 0.106, 0.101, 0.0...","[0.186, 0.155, 0.142, 0.135, 0.13, 0.125, 0.12...","[0.166, 0.133, 0.121, 0.112, 0.106, 0.101, 0.0...","[0.166, 0.133, 0.121, 0.112, 0.106, 0.101, 0.0..."
"Q-Former and LLM LoRA (all, FlanT5-XL)","[0.155, 0.129, 0.12, 0.113, 0.11, 0.103, 0.1, ...","[0.154, 0.131, 0.118, 0.237, 0.105, 0.101, 0.0...","[0.155, 0.126, 0.117, 0.111, 0.104, 0.099, 0.0...","[0.154, 0.13, 0.123, 0.112, 0.107, 0.101, 0.09..."
"LLM LoRA (ffn, Vicuna-7B)",0.0,0.0,0.0,0.0
"LLM LoRA (attn, Vicuna-7B)",0.0,0.0,0.0,0.0


In [78]:
val_acc_per_epoch_df.to_csv(f'{dataset}/val_acc_per_epoch.csv', index=True)
val_acc_per_epoch_df

,r = 1,r = 2,r = 4,r = 8
"LLM LoRA (ffn, FlanT5-XL)","[66.03863204559848, 68.84103863204561, 68.5877...","[67.19442685243826, 68.1760607979734, 69.49018...","[66.07029765674477, 66.2444585180494, 67.47941...","[66.2444585180494, 67.30525649145028, 70.55098..."
"LLM LoRA (attn, FlanT5-XL)","[63.01456618112729, 67.32108929702343, 68.0018...","[65.83280557314755, 67.59024699176695, 69.5851...","[64.70867637745408, 67.25775807473084, 68.2552...","[65.84863837872071, 67.2102596580114, 68.90436..."
"LLM LoRA (all, FlanT5-XL)","[66.95693476884104, 66.35528815706144, 70.4243...","[67.79607346421786, 68.99936668777707, 68.8568...","[66.56111462951235, 68.09689677010766, 71.1526...","[65.880303989867, 67.97023432552248, 68.936035..."
"Q-Former LoRA (ffn, FlanT5-XL)","[65.95946801773275, 65.94363521215959, 66.3711...","[65.95946801773275, 65.94363521215959, 66.3711...","[60.76630778974034, 63.220392653578216, 64.075...","[65.95946801773275, 65.94363521215959, 66.3711..."
"Q-Former LoRA (self-attn, FlanT5-XL)","[64.56618112729575, 66.5769474350855, 66.37112...","[64.56618112729575, 66.5769474350855, 66.37112...","[64.56618112729575, 66.5769474350855, 66.37112...","[64.56618112729575, 66.5769474350855, 66.37112..."
"Q-Former LoRA (cross-attn, FlanT5-XL)","[63.23622545915136, 53.79987333755542, 64.9620...","[63.23622545915136, 53.79987333755542, 64.9620...","[63.23622545915136, 53.79987333755542, 64.9620...","[63.23622545915136, 53.79987333755542, 64.9620..."
"Q-Former LoRA (all, FlanT5-XL)","[65.12032932235591, 66.8777707409753, 67.84357...","[60.148828372387584, 63.91703609879671, 63.790...","[65.12032932235591, 66.8777707409753, 67.84357...","[65.12032932235591, 66.8777707409753, 67.84357..."
"Q-Former and LLM LoRA (all, FlanT5-XL)","[65.83280557314755, 64.66117796073463, 65.5636...","[65.50031665611147, 64.48701709943002, 63.5845...","[62.127929069031026, 63.42621912602914, 65.373...","[62.365421152628244, 62.55541481950602, 67.463..."
"LLM LoRA (ffn, Vicuna-7B)",0.0,0.0,0.0,0.0
"LLM LoRA (attn, Vicuna-7B)",0.0,0.0,0.0,0.0
